<a href="https://colab.research.google.com/github/Krish-Ramoliya/DataScience_Projects/blob/main/WildFireNet_AI_Powered_Wildfire_Detection_with_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/elmadafri/the-wildfire-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: learner8969
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/elmadafri/the-wildfire-dataset


100%|██████████| 9.94G/9.94G [01:49<00:00, 97.6MB/s]


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os
from PIL import ImageFile

In [22]:
# Avoid Decompression Bomb Warning
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [16]:
# Define the paths to your dataset
train_dir = '/content/the-wildfire-dataset/the_wildfire_dataset_2n_version/train'
valid_dir = '/content/the-wildfire-dataset/the_wildfire_dataset_2n_version/val'
test_dir = '/content/the-wildfire-dataset/the_wildfire_dataset_2n_version/test'

In [6]:
# Image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

In [7]:
# Class weights to handle class imbalance
class_weights = {0: 1.0, 1: 1.58}  # Adjusted based on class imbalance

In [8]:
# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [17]:
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:


# Load dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Assuming binary classification (fire vs. no fire)
    shuffle=True
)


Found 1887 images belonging to 2 classes.


In [10]:
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 402 images belonging to 2 classes.


In [18]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 410 images belonging to 2 classes.


In [11]:
# Load MobileNetV2 model
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model layers


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])


In [13]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model with class weights
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,  # Adjust epochs as needed
    class_weight=class_weights
)



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
36/59 ━━━━━━━━━━━━━━━━━━━━ 3:14 8s/step - accuracy: 0.6198 - loss: 0.9600

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


40/59 ━━━━━━━━━━━━━━━━━━━━ 2:41 9s/step - accuracy: 0.6285 - loss: 0.9355

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


59/59 ━━━━━━━━━━━━━━━━━━━━ 635s 11s/step - accuracy: 0.6606 - loss: 0.8512 - val_accuracy: 0.8184 - val_loss: 0.4127
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 630s 11s/step - accuracy: 0.7949 - loss: 0.5289 - val_accuracy: 0.7687 - val_loss: 0.5015
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 640s 11s/step - accuracy: 0.8187 - loss: 0.4578 - val_accuracy: 0.8109 - val_loss: 0.4161
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 627s 11s/step - accuracy: 0.8370 - loss: 0.4437 - val_accuracy: 0.8408 - val_loss: 0.3921
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 628s 11s/step - accuracy: 0.8251 - loss: 0.4487 - val_accuracy: 0.8109 - val_loss: 0.4654
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 622s 11s/step - accuracy: 0.8341 - loss: 0.3938 - val_accuracy: 0.8458 - val_loss: 0.3476
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 598s 10s/step - accuracy: 0.8444 - loss: 0.4056 - val_accuracy: 0.8234 - val_loss: 0.3916
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 632s 11s/step - accuracy: 0.8655 - loss: 0.3944 - val_accuracy: 0.8109 - val_

In [15]:
# Save the model
model.save('wildfire_mobilenet_model.h5')

In [19]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

 4/13 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.6556 - loss: 0.8092 

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


 5/13 ━━━━━━━━━━━━━━━━━━━━ 51s 6s/step - accuracy: 0.6482 - loss: 0.8308

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 120s 9s/step - accuracy: 0.7388 - loss: 0.6331
Test Accuracy: 84.39%
